In [1]:
import cv2
import os
import numpy as np
from PIL import Image

In [2]:
with open("dataset/Videos/data_test1.rgb", "rb") as f:
    v = np.fromfile(f, np.dtype('B'))
#setting the number of pixels in frame
pxct = 129600
#setting the number of frames
frct = (int)(v.size/(3*pxct))

In [3]:
init_video = []
for frame_number in range(frct):
        start = (frame_number) * pxct * 3
        curr_frame = v[start:start + 3 * pxct]
        final_frame = np.reshape(curr_frame, (480,270,3), order = 'F')
        final_frame = np.rot90(final_frame,3)
        final_frame = np.flip(final_frame,1)
        
        #searching these final frames for logos (new cell)
        init_video.append(final_frame)

In [4]:
#IMAGE RECOGNITION FOR BRANDS




In [5]:
from skimage.metrics import structural_similarity
frame_stats = np.zeros((9000, 2)).tolist()


In [6]:
#save beginning and end of cuts in number of frames
cuts_begin = []
cuts_end = []
#storing the advertisements to replace with
#indexed the same way as that of cuts_begin and cuts_end
#if not to be replaced, a blank line
replaced_advertisements = []

In [7]:
out = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (480, 270))
for i in range(frct - 1):
    out.write(init_video[i])
out.release()

In [8]:
#image negatives for training
# for f in range(frct - 1):
#     if f > 3000 or f < 1200:
#         data = Image.fromarray(init_video[f]).convert('RGB')
#         data.save('negative_images/{}.png'.format(f))

In [9]:
for frame_number in range(frct - 1):
    #calculate new shots with histogram
    #less means more similar
    hist1 = cv2.calcHist([init_video[frame_number]],[0],None,[256],[0,256])
    hist2 = cv2.calcHist([init_video[frame_number + 1]],[0],None,[256],[0,256])
    score=cv2.compareHist(hist1,hist2,cv2.HISTCMP_BHATTACHARYYA)
    
    frame_stats[frame_number][0] = score
    #store score for ssi
    #less means more similar
    #i1 = cv2.cvtColor(init_video[frame_number], cv2.COLOR_BGR2GRAY)
    #i2 = cv2.cvtColor(init_video[frame_number + 1], cv2.COLOR_BGR2GRAY)
    #score2 = structural_similarity(i1, i2)
    #score2 = 1-score2 
    #frame_stats[frame_number][1] = score2
    
    #score indicates sensitivity for shot changes
    if score > 0.3:
        #saves the image, currently used for debugging
        #if score + score2 > 0.3:
        data = Image.fromarray(init_video[frame_number]).convert('RGB')
        data.save('images/{}.png'.format(frame_number))
        data = Image.fromarray(init_video[frame_number+1]).convert('RGB')
        data.save('images/{}.png'.format(frame_number+1))

In [10]:
#reading in all other advertisements
#saves in a dictionary with first part assumed to be the company name
advertisements = {}
PATHS_TO_ADVERTISEMENTS = "dataset/Ads"
for file in os.listdir(PATHS_TO_ADVERTISEMENTS):
    filename = os.fsdecode(file)
    if filename.endswith(".rgb"): 
        vpath = os.path.join(PATHS_TO_ADVERTISEMENTS, filename)
        with open(vpath, "rb") as f:
            v = np.fromfile(f, np.dtype('B'))
        #setting the number of pixels in frame
        pxct = 129600
        #setting the number of frames
        frct = (int)(v.size/(3*pxct))
        video = []
        for frame_number in range(frct):
            start = (frame_number) * pxct * 3
            curr_frame = v[start:start + 3 * pxct]
            final_frame = np.reshape(curr_frame, ( 480,270, 3), order = 'F')
            final_frame = np.rot90(final_frame,3)
            final_frame = np.flip(final_frame,1)
            video.append(final_frame)
        #Get the name of the company
        key = filename.split('_')[0]
        advertisements[key] = video
        
print(advertisements.keys())
    

dict_keys(['Starbucks', 'Subway'])


In [11]:
#splicing in videos with the names 
#save beginning and end of cuts in number of frames
cuts_begin = [2400]
cuts_end = [2849]
#storing the advertisements to replace with
#indexed the same way as that of cuts_begin and cuts_end
#if not to be replaced, an empty string
replaced_advertisements = ["Subway"]
final_video = []
for i in range (len(cuts_begin)):
    if i == 0:
        final_video = init_video[:cuts_begin[i]] + advertisements[replaced_advertisements[i]] + init_video[cuts_end[i]:]
    else:
        #further work required to sync frames iff ad lengths are variable
        final_video = final_video[:cuts_begin[i]] + advertisements[replaced_advertisements[i]] + final_video[cuts_end[i]:]
        

In [12]:
#trying with orb
#read in frame from video
# frame = Image.fromarray(final_video[2100]).convert('RGB')
# frame.save('images/box-test.png'.format(1));
# frame1 = cv2.imread('images/box-test.png')
# grayframe = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
# keypoints = []
# descriptors = []

# orb = cv2.ORB_create()

# kp1, des1 = orb.detectAndCompute(grayframe,None)

# files = os.listdir('augmentedData/')
# #add keypoints and descriptors from all images ("training" descriptor)
# for f in files:
#     try:
#         logo = cv2.imread('augmentedData/'+f)
#         (kps, descs) = detector.detectAndCompute(logo, None)
#         keypoints.append(kps)
#         descriptors.append(descs)
#         #brand.append() - need to add associated brand name here
#     except:
#         continue
    
# bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# matches = bf.match(des1, descriptors, k=2)
# print(matches)

# good_matches = []
# for m,n in matches:
#     if m.distance < 0.75*n.distance:
#         good_matches.append([m])
        
# print(good_matches)


In [13]:
# #classification process
# detector = cv2.SIFT_create()
# keypoints = []
# descriptors = []
# brand = []

# files = os.listdir('augmentedData/')
# #add keypoints and descriptors from all images ("training" descriptor)
# for f in files:
#     try:
#         logo = cv2.imread('augmentedData/'+f)
#         (kps, descs) = detector.detectAndCompute(logo, None)
#         keypoints.append(kps)
#         descriptors.append(descs)
#         #brand.append() - need to add associated brand name here
#     except:
#         continue
    
# # for f in files:
# #     try:
# #         logo = cv2.imread('logos/'+f)
# #         bwLogo = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
# #         (kps, descs) = detector.detectAndCompute(bwLogo, None)
# #         keypoints.append(kps)
# #         descriptors.append(descs)
# #     except:
# #         continue

# # # #read in frame from video - need to do for each frame!
# frame = Image.fromarray(final_video[2100]).convert('RGB')

# # # #reformat
# frame.save('images/box-test.png'.format(1));
# frame1 = cv2.imread('images/box-test.png')

# # # #change to grayscale
# grayframe = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    
# for d,kp in zip(descriptors, keypoints):
#     bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

#     matches = bf.knnMmatch(logodescriptors,framedescriptors)
#     matches = sorted(matches, key = lambda x:x.distance)
#     print("MATCHES ", matches)

        
    
    

In [14]:
#experimenting with SIFT
#finding points and comparing across image 
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

#read in logo 
logo = plt.imread('Dataset/Brand Images/subway_logo.bmp')
graylogo = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
#cv2.imshow('image', graylogo)
#cv2.waitKey(5000)

# #read in frame from video
frame = Image.fromarray(final_video[2100]).convert('RGB')

# #reformat
frame.save('images/box-test.png'.format(1));
frame1 = cv2.imread('images/box-test.png')

# #change to grayscale
grayframe = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT_create()

logopoints, logodescriptors = sift.detectAndCompute(graylogo, None)
framepoints, framedescriptors = sift.detectAndCompute(grayframe, None)

bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)
#bf = cv2.BFMatcher()

matches = bf.match(logodescriptors,framedescriptors)
#matches = bf.knnMatch(logodescriptors,framedescriptors, k=2)

matches = sorted(matches, key = lambda x:x.distance)

# Apply ratio test
#good_points = []
#for m,n in matches:
#    if m.distance < 0.7*n.distance:
#        good_points.append(m)
#print(good_points)

# cv2.drawMatchesKnn expects list of lists as matches.
#img3 = cv2.drawMatchesKnn(logo, logopoints, frame1, framepoints, good_points, None,
                              #matchColor=(0, 255, 0), matchesMask=None,
                              #singlePointColor=(255, 0, 0), flags=0)

#plt.imshow(img3),plt.show()



In [ ]:
#creating transform matrix to draw binding box
good_points = matches[:10]
src_pts = np.float32([ logopoints[m.queryIdx].pt for m in good_points]).reshape(-1,1,2)
dst_pts = np.float32([ framepoints[m.trainIdx].pt for m in good_points]).reshape(-1,1,2)
M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
matchesMask = mask.ravel().tolist()
h,w = logo.shape[:2]
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

dst = cv2.perspectiveTransform(pts,M)
dst += (w, 0)  # adding offset

draw_params = dict(matchColor = (0,255,0), # draw matches in green color
               singlePointColor = None,
               matchesMask = matchesMask, # draw only inliers
               flags = 2)

img3 = cv2.drawMatches(logo,logopoints,frame1,framepoints,good_points, None,**draw_params)

# Draw bounding box in Red
img3 = cv2.polylines(img3, [np.int32(dst)], True, (0,0,255),3, cv2.LINE_AA)

cv2.imshow("result", img3)
#cv2.waitKey()
# or another option for display output
#plt.imshow(img3, 'result'), plt.show(

In [ ]:
#create the video
#file path, format, frames per second, width and height
out = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (480, 270))
#RGBA format
frames = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]    

for i in range(len(final_video)):
    #cv2.imshow('images/box-1.png', img)
    data = Image.fromarray(final_video[i]).convert('RGB')
#print(type(data))

#convert rgb frame to png for writing
    data.save('images/box-test.png'.format(1));

#get new saved png frame
    img = cv2.imread('images/box-test.png')

    if(i in frames):
#draw rectangle on frame
        cv2.rectangle(img, (10,10), (100, 100), (0, 255, 0), 2)
#just to verify box comes out in right spot
        cv2.imwrite("images/test-box.png", img)

    #change back to rgb for avi video
        #img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        out.write(img)
    else:
        out.write(img)

out.release()

In [ ]:
#add function to find points and draw the box for each frame in the video
#output video to rgb
#

In [ ]:
#create more images to test on
# -*- coding: utf-8 -*-

#basic code taken from https://github.com/synckey/tensorflow_lstm_ctc_ocr/blob/master/gen_no_plate_shape_version.py
# """
# Created on Sat Dec 24 09:37:01 2016
# @author: shubham
# """
# import random
# import cv2
# import os
# import numpy
# import math
# num_images = 2700

# def euler_to_mat(yaw, pitch, roll):
#     # Rotate clockwise about the Y-axis
#     c, s = math.cos(yaw), math.sin(yaw)
#     M = numpy.matrix([[  c, 0.,  s],
#                       [ 0., 1., 0.],
#                       [ -s, 0.,  c]])

#     # Rotate clockwise about the X-axis
#     c, s = math.cos(pitch), math.sin(pitch)
#     M = numpy.matrix([[ 1., 0., 0.],
#                       [ 0.,  c, -s],
#                       [ 0.,  s,  c]]) * M

#     # Rotate clockwise about the Z-axis
#     c, s = math.cos(roll), math.sin(roll)
#     M = numpy.matrix([[  c, -s, 0.],
#                       [  s,  c, 0.],
#                       [ 0., 0., 1.]]) * M

#     return M

# def make_affine_transform(from_shape, to_shape, 
#                           min_scale, max_scale,
#                           scale_variation=1.0,
#                           rotation_variation=1.0,
#                           translation_variation=1.0):
#     out_of_bounds = False

#     from_size = numpy.array([[from_shape[1], from_shape[0]]]).T
#     to_size = numpy.array([[to_shape[1], to_shape[0]]]).T

#     scale = random.uniform((min_scale + max_scale) * 0.5 -
#                            (max_scale - min_scale) * 0.5 * scale_variation,
#                            (min_scale + max_scale) * 0.5 +
#                            (max_scale - min_scale) * 0.5 * scale_variation)
#     if scale > max_scale or scale < min_scale:
#         out_of_bounds = True
#     roll = random.uniform(-0.3, 0.3) * rotation_variation
#     pitch = random.uniform(-0.2, 0.2) * rotation_variation
#     yaw = random.uniform(-1.2, 1.2) * rotation_variation

#     # Compute a bounding box on the skewed input image (`from_shape`).
#     M = euler_to_mat(yaw, pitch, roll)[:2, :2]
#     h, w = from_shape
#     corners = numpy.matrix([[-w, +w, -w, +w],
#                             [-h, -h, +h, +h]]) * 0.5
#     skewed_size = numpy.array(numpy.max(M * corners, axis=1) -
#                               numpy.min(M * corners, axis=1))

#     scale *= numpy.min(to_size / skewed_size)

#     trans = (numpy.random.random((2,1)) - 0.5) * translation_variation
#     trans = ((2.0 * trans) ** 5.0) / 2.0
#     if numpy.any(trans < -0.5) or numpy.any(trans > 0.5):
#         out_of_bounds = True
#     trans = (to_size - skewed_size * scale) * trans

#     center_to = to_size / 2.
#     center_from = from_size / 2.

#     M = euler_to_mat(yaw, pitch, roll)[:2, :2]
#     M *= scale
#     M = numpy.hstack([M, trans + center_to - M * center_from])

#     return M, out_of_bounds


# for i in range(0,num_images):
#     f=random.choice(os.listdir('logos/'))
#     logoI = plt.imread('logos/'+f)
#     logo = cv2.cvtColor(logoI, cv2.COLOR_BGR2GRAY)   
    
#     scale = 1;#random.randint(0,5)
#     if scale!=0 and ((logo.shape[0]/scale)>10):  
#         M, out_of_bounds = make_affine_transform(
#                             from_shape=logo.shape,
#                             to_shape=logo.shape,
#                             min_scale=1,
#                             max_scale=1,
#                             rotation_variation=1.0,
#                             scale_variation=1.5,
#                             translation_variation=1.2)   
                            
#         im = cv2.warpAffine(logo,M,(500, 255))
#                             #logo.shape,255)

#         cv2.imwrite('augmentedData/'+str(i)+'.png', im)
#     else:
#         cv2.imwrite('augmentedData/'+str(i)+'.png', im)